# Imports 

In [1]:
import requests
import glob, shutil, os
import numpy as np
import pandas as pd

# Parameters
__You can adapt any parameters found in this block__.

BnF corpus refers to Bibliothèque nationale de France's Gallica

In [2]:
# URL prefix
prefix = 'https://gallica.bnf.fr/iiif/ark:/12148/'

# Load dataframe

In [3]:
df = pd.read_json('data/df_maps_BnF.json').reset_index().drop(columns = 'index')

In [4]:
df.head(3)

,coverage,date,description,identifier,language,publisher,relation,subject,source,format,title,scale,ark,geolocalisation,n_items,latitude,longitude,filesnames,folds
0,France – Île-de-France – Paris,1937.0,Échelle(s) : Echelle:1:17 000,https://gallica.bnf.fr/ark:/12148/btv1b531212347,fre,impr. Dufrénoy (Paris),Notice du catalogue : http://catalogue.bnf.fr/...,Paris,"Bibliothèque nationale de France, département ...","[1 flle : en bistre ; 92 x 60 cm, image/jpeg, ...",Paris,17000.0,btv1b531212347,,1,NaN,NaN,maps/BnF/12148_btv1b531212347f1.jpg,[0]
1,France – Île-de-France – Paris,1944.0,Échelle(s) : 1:17 500,https://gallica.bnf.fr/ark:/12148/btv1b53121232b,fre,Girard et Barrère (Paris),Notice du catalogue : http://catalogue.bnf.fr/...,Paris -- Plans de ville,"Bibliothèque nationale de France, département ...","[1 flle : en coul. ; 82 x 60 cm, image/jpeg, N...",Paris,17500.0,btv1b53121232b,,1,NaN,NaN,maps/BnF/12148_btv1b53121232bf1.jpg,[0]
2,France – Île-de-France – Paris,1829.0,,https://gallica.bnf.fr/ark:/12148/btv1b8492750g,fre,A.P.Levalton (Paris),[Notice de recueil : http://catalogue.bnf.fr/a...,"[Plans de ville, Paris]","Bibliothèque nationale de France, département ...","[1 flle ; 640 x 470 avec tracés en coul., imag...",Plan de Paris en 1829 avec le tracé des ancien...,NaN,btv1b8492750g,,1,NaN,NaN,maps/BnF/12148_btv1b8492750gf1.jpg,[0]


In [5]:
len(df)

773

# Download images with IIIF (Gallica API)
Can only works for BnF images !

In [6]:
region = 'full'
size = 'full'
rotation = '0'
quality = 'native'
imgformat = 'jpg'
path = 'maps/'

In [ ]:
present_files = glob.glob(os.path.join(path, '*.jpg'))

for ind, row in df[['identifier', 'folds', 'ark']].iterrows():
    
    for page in row['folds']:
        leaflet = 'f' + str(int(page)+1)
        url = "".join([prefix, row['ark'], '/', leaflet, '/', 
                       region, '/', size, '/', rotation, '/', quality, '.', imgformat])
        
        filename = "".join([path + row['ark'], leaflet, '.', imgformat])

        dirname = os.path.dirname(filename)
        if not os.path.exists(dirname):
            os.makedirs(dirname)
            
        if not(filename in present_files): 
            response = requests.get(url, stream=True)
            
            with open(filename, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
                del response